# Дашборд "Экспорт минеральных удобрений в 2018-2019 гг."

Компания-трейдер, занимающаяся промышленными химикатами, приобрела данные статистики ВЭД у консалтинговой фирмы.  
Сведения помогут лучше понимать конъюнктуру рынка и выработать торговую стратегию. 

"Сырые" табличные данные сложны для восприятия и не годятся для быстрого анализа.  
Руководитель отдела продаж просит подготовить информацию в удобном виде с возможнностью получать срезы.

**Задача:**  

Агрегировать и визуализировать данные таможенной статистики на дашборде с доступом по ссылке.   
Обязательно отразить объёмы и цены, ключевые наравления экспорта.  

**Бизнес согласовал следующие срезы данных:**
- период (годы) 
- товарная подгруппа (вид удобрений)
- регион поставки 
- тип перевозки (в контейнере или нет)  

Дашборд доступен по ссылке  
https://datalens.yandex/o8pv5nlhk1wcc 

## План работы

1. Загрузить и проверить данные

2. Подготовить файл-витрину данных для передачи в дашборд
    - объединить файлы
    - определить перечень необходимых полей и сократить данные
    - подготовить выборку, соответствующую задаче (только экспорт, факт. сделки и т.д.)
    - сделать группировку стран по макрорегионам

3. Подготовить дашборд на облачной платформе DataLens


## Описание данных

Данные находятся в файлах `fertilizers_2018.csv` и `fertilizers_2019.csv`.  
Данные за 2 полных года.

Имеется множество столбцов, необходимо выделить нужные.  
Заказчик сообщил, что:
- его интересует только экспорт 
- только подтверждённые декларации (поле `STAT`=1)
- приведённые цены в USD (использовать поле `G46 (Статистическая стоимость, USD.)`)

### Подключение библиотек

In [1]:
import pandas as pd
import numpy as np

### Загрузка данных

In [2]:
df_2018 = pd.read_csv('fertilizers_2018.csv', sep=';', low_memory=False)
df_2019 = pd.read_csv('fertilizers_2019.csv', sep=';', low_memory=False)

In [3]:
df_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19449 entries, 0 to 19448
Columns: 107 entries, ND (Номер декларации) to FIRM (Доп.информация о контрактодержателе (Росстат))
dtypes: float64(11), int64(19), object(77)
memory usage: 15.9+ MB


In [4]:
df_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21670 entries, 0 to 21669
Columns: 107 entries, ND (Номер декларации) to FIRM (Доп.информация о контрактодержателе (Росстат))
dtypes: float64(19), int64(11), object(77)
memory usage: 17.7+ MB


In [5]:
print("Всего строк в двух файлах:", df_2018.shape[0] + df_2019.shape[0])

Всего строк в двух файлах: 41119


- Всего 41 тыс. строк - отлично, это ниже лимита построения графиков на Data Lens (до 75 тыс. строк)   
https://datalens.tech/docs/ru/concepts/limits.html

- Полей 107 одининаковое кол-во в обоих файлах. Проверим, идентичны ли названия полей

- 107 полей много для нашей задачи, выберем только необходимые

### Подготовка файла-витрины для передачи в дашборд

#### Объединение файлов, выбор нужных полей
Проверим, совпадают ли именая полей в файлах.  
Если да, можно сразу объединять.  

In [6]:
# сравним 2 множества из названий полей

diff_colls_quantity = len(set(df_2018.columns).difference(df_2019.columns))

if diff_colls_quantity == 0:
    print("Все названия полей совпали, можно объединять")
else:
    print("Есть не совпадающие названия полей, нужно разобраться перед объединением")

Все названия полей совпали, можно объединять


In [7]:
df_all = pd.concat([df_2018, df_2019], axis=0)

In [8]:
# Объединение корректно, можно удалить исходные датасеты из ОЗУ

df_all.shape

del df_2019, df_2018, diff_colls_quantity

Посмотрим на список полей. Оставим только нужные для решения нашей задачи.  
Для удобства можно записать список в отдельный текстовый файл и посмотреть отдельно.

In [9]:
pd.DataFrame(df_all.columns).to_csv('columns.txt', sep=' ', index=False)

In [10]:
# Создадим перечень необходимых полей

selected_cols = [
'STAT (STAT)',
'GD1 (Дата выпуска)',
'G011 (Направление перемещения)',
'G17B (Страна назначения)',
'G17A (Код страны назначения)',
'G19 (Признак контейнерных перевозок)',
'G38 (Вес нетто, кг)',
'G46 (Статистическая стоимость, USD.)',
'G33 (Код товара по ТН ВЭД)',
"G083 (Адрес получателя)",
'G0831 (Код страны получателя)'
]

In [11]:
df_all = df_all[selected_cols]

In [12]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41119 entries, 0 to 21669
Data columns (total 11 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   STAT (STAT)                           41119 non-null  int64  
 1   GD1 (Дата выпуска)                    41119 non-null  object 
 2   G011 (Направление перемещения)        41119 non-null  object 
 3   G17B (Страна назначения)              40925 non-null  object 
 4   G17A (Код страны назначения)          40767 non-null  object 
 5   G19 (Признак контейнерных перевозок)  38484 non-null  float64
 6   G38 (Вес нетто, кг)                   41119 non-null  object 
 7   G46 (Статистическая стоимость, USD.)  41119 non-null  object 
 8   G33 (Код товара по ТН ВЭД)            41119 non-null  object 
 9   G083 (Адрес получателя)               41118 non-null  object 
 10  G0831 (Код страны получателя)         38336 non-null  object 
dtypes: float64(1), int64

In [13]:
df_all.head()

,STAT (STAT),GD1 (Дата выпуска),G011 (Направление перемещения),G17B (Страна назначения),G17A (Код страны назначения),G19 (Признак контейнерных перевозок),"G38 (Вес нетто, кг)","G46 (Статистическая стоимость, USD.)",G33 (Код товара по ТН ВЭД),G083 (Адрес получателя),G0831 (Код страны получателя)
0,0,02.01.2018,ЭК,ГЕРМАНИЯ,DE,0.0,5000000,1078600,3102101000,"238871, PALAIS RENAISSANCE, SINGAPORE, 08-04 O...",DE
1,1,03.01.2018,ЭК,БРАЗИЛИЯ,BR,0.0,1611000,565461,3105400000,"LV-3602, , Г.ВЕНТСПИЛС, УЛ. ДЗИНТАРУ, 41",BR
2,1,03.01.2018,ЭК,ВЕНГРИЯ,HU,0.0,1020000,389640,3105400000,"2943, , БАБОЛНА, ИКР ПАРК № 890",HU
3,1,03.01.2018,ЭК,ЛАТВИЯ,LV,0.0,960000,343680,3105400000,"LV-1004, , РИГА, УЛ.БАУСКАС 58А-13",LV
4,1,03.01.2018,ЭК,ЛИТВА,LT,0.0,512000,189440,3105400000,"12126, , Г. ВИЛЬНЮС, УЛ. ПЕРКУНКИЕМИО, 2",LT


Данные объединены и выбраны корректно, но есть пропуски:
- В названии страны назначения и кодах с ней связанных.
- В поле-флаге, обозначающем контейнерные перевозки.

### Предобработка данных

Отлично, есть только необходные данные. Теперь нужно:
 
- Выбрать только экспорт + действительные декларации, у которых поле `STAT`=1

- Заполнить пропуски в поле по признаку контейнерных перевозок. Если данных нет, то считаем, что груз едет не в контейнере.
  Бизнес заказчик согласовал и уточнил, что в контейнерах должно быть гораздо меньше перевозок.

- Обработать поля страны назначения и добавить агрегирование по региону.

- Добавить геоданные, чтобы иметь возможность отображать ключевые регионы поставок на карте.

In [14]:
# Посмотрим как обозначенны направления перемещения

df_all['G011 (Направление перемещения)'].value_counts()

G011 (Направление перемещения)
ЭК    33910
ИМ     7209
Name: count, dtype: int64

In [15]:
# Сделаем подвыборку согласно задаче и удалим поля

df_all = df_all.loc[(df_all['STAT (STAT)'] == 1) & (df_all['G011 (Направление перемещения)'] == 'ЭК')]
df_all = df_all.drop(columns=['STAT (STAT)', 'G011 (Направление перемещения)'])

In [16]:
df_all['G19 (Признак контейнерных перевозок)'].value_counts()

G19 (Признак контейнерных перевозок)
0.0    20508
1.0     2757
Name: count, dtype: int64

In [17]:
# Заполним пропуски в поле контейнерных перевозок флагом, что это не контейнеры.

df_all['G19 (Признак контейнерных перевозок)'] = df_all['G19 (Признак контейнерных перевозок)'].fillna(0)

In [18]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24985 entries, 1 to 21668
Data columns (total 9 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   GD1 (Дата выпуска)                    24985 non-null  object 
 1   G17B (Страна назначения)              24967 non-null  object 
 2   G17A (Код страны назначения)          24954 non-null  object 
 3   G19 (Признак контейнерных перевозок)  24985 non-null  float64
 4   G38 (Вес нетто, кг)                   24985 non-null  object 
 5   G46 (Статистическая стоимость, USD.)  24985 non-null  object 
 6   G33 (Код товара по ТН ВЭД)            24985 non-null  object 
 7   G083 (Адрес получателя)               24984 non-null  object 
 8   G0831 (Код страны получателя)         22986 non-null  object 
dtypes: float64(1), object(8)
memory usage: 1.9+ MB


#### Страны и регионы поставок
- определим, какое поле лучше брать для группировки стран (код или название)
- сгрупируем страны по регионам, используя уже доступную классификацию



##### Обработка пропусков

In [19]:
print('Всего уникальных стран по названию в наших данных:', len(df_all['G17B (Страна назначения)'].unique()))
print('Всего уникальных стран по буквенному коду в наших данных:', len(df_all['G17A (Код страны назначения)'].unique()))

Всего уникальных стран по названию в наших данных: 141
Всего уникальных стран по буквенному коду в наших данных: 135


Пропуски есть в кодах и названиях стран. Число уникальных кодов и названий не бьётся, значит:
- либо у некоторых стран по-разному написаны названия
- или не у каждой страны есть код/название 

In [20]:
df_countries = df_all[['G17B (Страна назначения)', 'G17A (Код страны назначения)']].copy()

df_countries['key'] = df_all['G17B (Страна назначения)'] + '|' + df_all['G17A (Код страны назначения)']
df_countries.set_index('G17A (Код страны назначения)')
df_countries.drop(columns='G17B (Страна назначения)', inplace=True)

df_countries.drop_duplicates(inplace=True)
df_countries.value_counts(dropna=False).head()


G17A (Код страны назначения)  key              
AB                            АБХАЗИЯ|AB           1
NZ                            НОВАЯ ЗЕЛАНДИЯ|NZ    1
MY                            МАЛАЙЗИЯ|MY          1
MZ                            МОЗАМБИК|MZ          1
NG                            НИГЕРИЯ|NG           1
Name: count, dtype: int64

Нет кодов стран с 2 и более уникальными ключами название+код, названия одной и той же страны единообразны.  

Значит дело в пропусках в кодах и/или названиях стран.

In [21]:
# Посмотрим на страны, у которых не указано название или код

df_countries = df_all[['G17B (Страна назначения)', 'G17A (Код страны назначения)']].copy()
df_countries.loc[(df_countries['G17A (Код страны назначения)'].isna())|(df_countries['G17A (Код страны назначения)'].isna())].drop_duplicates()

,G17B (Страна назначения),G17A (Код страны назначения)
322,НАМИБИЯ,NaN
17328,СЕВЕРНАЯ МАКЕДОНИЯ,NaN
18378,NaN,NaN


In [22]:
df_countries.loc[(df_countries['G17B (Страна назначения)'] =='НАМИБИЯ')|(df_countries['G17B (Страна назначения)'] =='СЕВЕРНАЯ МАКЕДОНИЯ')]

,G17B (Страна назначения),G17A (Код страны назначения)
322,НАМИБИЯ,NaN
348,НАМИБИЯ,NaN
859,НАМИБИЯ,NaN
7802,НАМИБИЯ,NaN
12074,НАМИБИЯ,NaN
13024,НАМИБИЯ,NaN
13265,НАМИБИЯ,NaN
6406,НАМИБИЯ,NaN
6482,НАМИБИЯ,NaN
14271,НАМИБИЯ,NaN


Пропущены коды для Намибии и Северной Македонии. Заполним их.

In [23]:
df_all.loc[df_all['G17B (Страна назначения)']=='НАМИБИЯ', 'G17A (Код страны назначения)'] = 'NA'
df_all.loc[df_all['G17B (Страна назначения)']=='СЕВЕРНАЯ МАКЕДОНИЯ', 'G17A (Код страны назначения)'] = 'MK'

Посмотрим на оставшиеся пропуски.  
М.б. поля "G0831 (Код страны получателя)" и "G083 (Адрес получателя)" помут понять как заполнить пропуски.

In [24]:
df_all.loc[df_all['G17B (Страна назначения)'].isna(), ['G083 (Адрес получателя)', "G0831 (Код страны получателя)"]]

,G083 (Адрес получателя),G0831 (Код страны получателя)
18378,"6300, ZUG, GOTTHARDSTRASSE 2.",NaN
18412,"6300, ZUG, BAARERSTRASSE 37.",NaN
18413,"6300, ZUG, BAARERSTRASSE 37.",NaN
18414,"6300, ZUG, BAARERSTRASSE 37.",NaN
18430,"6300, ZUG, BAARERSTRASSE 37.",NaN
18431,"6300, ZUG, BAARERSTRASSE 37.",NaN
18432,"6300, ZUG, BAARERSTRASSE 37.",NaN
18433,"6300, ZUG, BAARERSTRASSE 37.",NaN
18434,"6300, ZUG, BAARERSTRASSE 37.",NaN
18767,"6300, ZUG, BAARERSTRASSE 37.",NaN


К сожалению, из этих данных нельзя точно установить страну назначения, т.к. это адреса трейдеров в Швейцарии.  
Пропусков очень мало, они не повлияют на общую картину. Отбросим их и избыточные поля. 

In [25]:
df_all = df_all.drop(columns=['G083 (Адрес получателя)', 'G0831 (Код страны получателя)'])
df_all = df_all.dropna()

##### Группировка по регионам  
- Подтянем нормализованные названия стран и региональную группировку из готового справочника.
- Если что-то не заполниится, дозаполним по смыслу.


In [26]:
# Возьмём уже готовый классификатор стран и регионов.

countries_and_regions = pd.read_csv('https://www.artlebedev.ru/country-list/tab/', sep='\t')

In [27]:
countries_and_regions.head()

,name,fullname,english,alpha2,alpha3,iso,location,location-precise
0,Абхазия,Республика Абхазия,Abkhazia,AB,ABH,895,Азия,Закавказье
1,Австралия,NaN,Australia,AU,AUS,36,Океания,Австралия и Новая Зеландия
2,Австрия,Австрийская Республика,Austria,AT,AUT,40,Европа,Западная Европа
3,Азербайджан,Республика Азербайджан,Azerbaijan,AZ,AZE,31,Азия,Западная Азия
4,Албания,Республика Албания,Albania,AL,ALB,8,Европа,Южная Европа


In [28]:
countries_and_regions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   name              251 non-null    object
 1   fullname          170 non-null    object
 2   english           251 non-null    object
 3   alpha2            250 non-null    object
 4   alpha3            251 non-null    object
 5   iso               251 non-null    int64 
 6   location          247 non-null    object
 7   location-precise  251 non-null    object
dtypes: int64(1), object(7)
memory usage: 15.8+ KB


Код альфа-2 какой-то страны не спарсился правильно. Проверим и исправим.

In [29]:
countries_and_regions.loc[countries_and_regions['alpha2'].isna(), 'name']

135    Намибия
Name: name, dtype: object

In [30]:
countries_and_regions.loc[countries_and_regions['name'] == 'Намибия', 'alpha2'] = 'NA'

In [31]:
# Переносим коды стран в индекс и объединяем таблицы

countries_and_regions = countries_and_regions.set_index('alpha2')
df_all = df_all.set_index('G17A (Код страны назначения)')

df_all = df_all.join(countries_and_regions)

In [32]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24967 entries, BR to ID
Data columns (total 13 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   GD1 (Дата выпуска)                    24967 non-null  object 
 1   G17B (Страна назначения)              24967 non-null  object 
 2   G19 (Признак контейнерных перевозок)  24967 non-null  float64
 3   G38 (Вес нетто, кг)                   24967 non-null  object 
 4   G46 (Статистическая стоимость, USD.)  24967 non-null  object 
 5   G33 (Код товара по ТН ВЭД)            24967 non-null  object 
 6   name                                  24967 non-null  object 
 7   fullname                              18074 non-null  object 
 8   english                               24967 non-null  object 
 9   alpha3                                24967 non-null  object 
 10  iso                                   24967 non-null  int64  
 11  location              

In [33]:
# Удалим ненужные поля и индекс, поправим имена

df_all = df_all.drop(columns=['fullname', 'alpha3', 'iso', 'G17B (Страна назначения)'])
df_all.rename(inplace=True, columns={'GD1 (Дата выпуска)':'date', 
                                     'G19 (Признак контейнерных перевозок)':'conteiner',
                                     'G38 (Вес нетто, кг)':'kg',
                                     'G46 (Статистическая стоимость, USD.)':'value_usd',
                                     'G33 (Код товара по ТН ВЭД)':'goods_code',
                                     'name':'country',
                                     'location':'macro_region',
                                     'location-precise':'region',
                                    })

df_all.reset_index(drop=True, inplace=True)

#### Товарные подгруппы
- Сделаем разбивку на крупные товарные подгруппы в зависимости от первых четырёх цифр кода товара.   
https://www.alta.ru/tnved/

In [34]:
# Урежем товарный код под нашу классификацию.

df_all['goods_code'] = df_all['goods_code'].str[0:4]

In [35]:
# Создадим классификацию товарных подгрупп, товарный код удалим

goods_dict = {'3102':'азотные',
              '3105':'сложные и комплексные',
              '3104':'калийные',
              '3103':'фосфорные', 
              '3101':'прочее'}

df_all['goods_type'] = df_all['goods_code'].map(goods_dict)

df_all.drop(columns='goods_code', inplace=True)

#### Подготовка геоданных
Подготовим геоданные по странам формате, оптимизированном под BI серис DataLens.  
Согласно документации, геоданные можно взять тут https://geointellect.com/files/geo_for_datalens.zip   

Данные по странам мира лежат в файле `all_country_borders.csv`  

К сожалению, если донасытить наши данные геоданными в обоих форматах (геополигоны и геоточки) выходной файл получается слишком большой (1 Гб).
- такой размер не возможно загрузить в DataLens
- так же не возможно объединить сдалать аналогочное объединение уже силами DataLens из 2 файлов - карта не строится.

Остановимся на более компактном виде геоданных - геоточках.

In [36]:
df_geo = pd.read_csv('all_country_borders.csv', sep=';')

In [37]:
df_geo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name_ru      442 non-null    object
 1   name_en      442 non-null    object
 2   coords_type  442 non-null    object
 3   coords       442 non-null    object
dtypes: object(4)
memory usage: 13.9+ KB


In [38]:
df_geo.sample(5)

,name_ru,name_en,coords_type,coords
303,Индонезия,Indonesia,all_country_borders_points,"[-2.81876721211,118.407966183]"
367,Малави,Malawi,all_country_borders_points,"[-13.4120466802,33.48353745]"
185,Кирибати,Kiribati,all_country_borders_poly,"[[[1.588628300000039,172.64367330000005],[1.88..."
35,Ливан,Lebanon,all_country_borders_poly,"[[[34.69215149999999,36.33979760000001],[34.68..."
416,Доминиканская Республика,Dominican Republic,all_country_borders_points,"[19.2936628131,-70.1996012724]"


In [39]:
# Формат таблицы с отдельным столбцом типа геоданных неудобный.   
# Преобразуем таблицу, чтобы типы геоточек были каждый в отдельном столбце.

df_geo = pd.crosstab(index=df_geo['name_en'], columns=df_geo['coords_type'], values=df_geo['coords'], aggfunc='sum')

In [40]:
df_geo.head()

coords_type,all_country_borders_points,all_country_borders_poly
name_en,,
Abkhazia,"[43.0478382899,41.0424398634]","[[[43.3855385,40.0085526],[43.4243519,40.01014..."
Afghanistan,"[33.8312497952,66.0274266802]","[[[36.925945,73.841253],[36.890824,73.815059],..."
Albania,"[41.08574876,19.9373854716]","[[[40.0606217,19.3057153],[40.1799986,19.16839..."
Algeria,"[28.2644117708,2.68192736543]","[[[35.4152605,8.3176717],[35.4046278,8.3297734..."
Andorra,"[42.5455453842,1.57621683525]","[[[42.603722799999986,1.4419909],[42.6130817,1..."


In [41]:
df_geo = df_geo.reset_index()

In [42]:
# Для объединения возьмём только геоточки. 

df_geo = df_geo[['name_en', 'all_country_borders_points']]

In [43]:
df_all = df_all.merge(df_geo, left_on='english', right_on='name_en', how='left')

## Проверка и выгрузка итогового дата фрейма

In [44]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24967 entries, 0 to 24966
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   date                        24967 non-null  object 
 1   conteiner                   24967 non-null  float64
 2   kg                          24967 non-null  object 
 3   value_usd                   24967 non-null  object 
 4   country                     24967 non-null  object 
 5   english                     24967 non-null  object 
 6   macro_region                24967 non-null  object 
 7   region                      24967 non-null  object 
 8   goods_type                  24967 non-null  object 
 9   name_en                     24967 non-null  object 
 10  all_country_borders_points  24967 non-null  object 
dtypes: float64(1), object(10)
memory usage: 2.1+ MB


In [45]:
df_all.head()

,date,conteiner,kg,value_usd,country,english,macro_region,region,goods_type,name_en,all_country_borders_points
0,03.01.2018,0.0,1611000,565461,Бразилия,Brazil,Америка,Южная Америка,сложные и комплексные,Brazil,"[-10.8279176405,-52.8483115037]"
1,03.01.2018,0.0,1020000,389640,Венгрия,Hungary,Европа,Восточная Европа,сложные и комплексные,Hungary,"[47.1652797826,19.4121477714]"
2,03.01.2018,0.0,960000,343680,Латвия,Latvia,Европа,Северная Европа,сложные и комплексные,Latvia,"[56.9091738905,24.5669304062]"
3,03.01.2018,0.0,512000,189440,Литва,Lithuania,Европа,Северная Европа,сложные и комплексные,Lithuania,"[55.347247978,23.8004638579]"
4,03.01.2018,0.0,320000,118400,Литва,Lithuania,Европа,Северная Европа,сложные и комплексные,Lithuania,"[55.347247978,23.8004638579]"


- [x] Данные предоброботаны, всё смэтчилось.  
- [x] Типы и разделители оставим как есть и обработаем уже в среде дашборда.

Можно выгружать в исходный файл для дашборда. `data_for_YL_dashboard.csv`  
Дашборд доступен по ссылке https://datalens.yandex/o8pv5nlhk1wcc 

In [46]:
df_all.to_csv('data_for_YL_dashboard.csv', sep=';', index=False)